# import

In [1]:
import pprint

from dotenv import load_dotenv

from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.documents import Document
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage



USER_AGENT environment variable not set, consider setting it to identify your requests.


# loading env variables

In [2]:
load_dotenv()

True

# Langain & Ollama

## check ollama status

In [3]:
!curl --location 'http://127.0.0.1:11434/api/generate' \
--header 'Content-Type: application/json' \
--data '{ \
    "model": "llama3.1:8b", \
    "prompt": "hello llama!",  \
    "stream": false, \
    "options": { \
        "temperature": 0 \
    } \
}' \
# | python -m json.tool

{"model":"llama3.1:8b","created_at":"2024-07-24T08:50:54.880841121Z","response":"Hello there, llama! What's up? Do you have any fun facts or questions to share? I'm all ears (or rather, all text)!","done":true,"done_reason":"stop","context":[128006,882,128007,271,15339,94776,0,128009,128006,78191,128007,271,9906,1070,11,94776,0,3639,596,709,30,3234,499,617,904,2523,13363,477,4860,311,4430,30,358,2846,682,25212,320,269,4856,11,682,1495,42395],"total_duration":5693293559,"load_duration":4901260413,"prompt_eval_count":12,"prompt_eval_duration":33592000,"eval_count":32,"eval_duration":756249000}

In [5]:
!curl http://localhost:11434/api/tags

{"models":[{"name":"starcoder2:7b","model":"starcoder2:7b","modified_at":"2024-07-24T01:16:23.996381581+02:00","size":4042812905,"digest":"0679cedc1189e4e55d25cc09459a75a3e753fb5d3ac9b3cb58269b645fd68456","details":{"parent_model":"","format":"gguf","family":"starcoder2","families":["starcoder2"],"parameter_size":"7B","quantization_level":"Q4_0"}},{"name":"llama3.1:70b","model":"llama3.1:70b","modified_at":"2024-07-24T00:40:45.706517802+02:00","size":39969745954,"digest":"fb41669f7289532b6d69b194a8969593713f3cf85304e1fb943fb73e66cb10db","details":{"parent_model":"","format":"gguf","family":"llama","families":["llama"],"parameter_size":"70.6B","quantization_level":"Q4_0"}},{"name":"llama3.1:8b","model":"llama3.1:8b","modified_at":"2024-07-24T00:14:48.766476389+02:00","size":4661225025,"digest":"a23da2a8039594d39ca68c68ea1c0ccf9155889e8197f7d17dd2743a1aee3f14","details":{"parent_model":"","format":"gguf","family":"llama","families":["llama"],"parameter_size":"8.0B","quantization_level":"

## Ollama model: configuration

In [6]:
llm = Ollama(model="llama3.1:8b", temperature=0)

## testing llm: invoke


In [7]:
# llm.invoke(input="tell me a joke")
response = llm.invoke("hello ollama!")
# response = llm.invoke("Create an agent that uses Ollama function calling in Langchain.")

print(response)

Hello! How can I assist you today?


## testing llm: chat prompt template

In [9]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
]) 

chain = chat_prompt_template | llm

response = chain.invoke({"input": "how can langsmith help with testing?"})

print(response)

As a world-class technical documentation writer, I'd be happy to explain how LangSmith can assist with testing.

LangSmith is an AI-powered language analysis tool that can help with testing in several ways:

1. **Grammar and syntax checking**: LangSmith's grammar and syntax checker can identify errors in your documentation, such as incorrect verb tenses, missing articles, or misplaced commas. This ensures that your content is error-free and easy to read.
2. **Style and consistency checks**: The tool can also check for style and consistency issues, like formatting inconsistencies, redundant phrases, or unclear sentence structures. This helps maintain a consistent tone and voice throughout your documentation.
3. **Readability analysis**: LangSmith's readability metrics can help you assess the complexity of your content, making it easier to identify areas that may need simplification or reorganization for better comprehension.
4. **Plagiarism detection**: The tool can also detect plagiari

## testing llm: chat prompt template & StrOutputParser

In [10]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
]) 

output_parser = StrOutputParser()

chain = chat_prompt_template | llm | output_parser

response = chain.invoke({"input": "how can langsmith help with testing?"})

print(response)

As a world-class technical documentation writer, I'd be happy to explain how LangSmith can assist with testing.

LangSmith is an AI-powered language analysis tool that can help with testing in several ways:

1. **Grammar and syntax checking**: LangSmith's grammar and syntax checker can identify errors in your documentation, such as incorrect verb tenses, missing articles, or misplaced commas. This ensures that your content is error-free and easy to read.
2. **Style and consistency checks**: The tool can also check for style and consistency issues, like formatting inconsistencies, redundant phrases, or unclear sentence structures. This helps maintain a consistent tone and voice throughout your documentation.
3. **Readability analysis**: LangSmith's readability metrics can help you assess the complexity of your content, making it easier to identify areas that may need simplification or reorganization for better comprehension.
4. **Plagiarism detection**: The tool can also detect plagiari

## create vector store & create a retriever

In [25]:
# 1. select a specfic datasource. In this case a web page. 
# 2. save extracted content from the web page as docs.
# 3. index the docs using FAISS vector store.
# 4. convert the vector store to retriever.

web_base_loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = web_base_loader.load()
# print(docs)
# print(type(docs))
# print(len(docs))
# type(docs[0])
# print(docs[0].page_content)

recursive_character_text_splitter = RecursiveCharacterTextSplitter()
documents = recursive_character_text_splitter.split_documents(documents=docs)

# print(type(documents))
# print(documents)
# print(len(documents))
# print(documents[0])
# print(documents[3])

ollama_embedding = OllamaEmbeddings(model="llama3:8b")
vector_store = FAISS.from_documents(documents=documents, embedding=ollama_embedding)

# vector_store.index.ntotal
# vector_store._get_retriever_tags()
# vector_store.index_to_docstore_id
# type(vector_store.index_to_docstore_id)

vector_store_retriever = vector_store.as_retriever()

## document chain

In [26]:
# 5. create a chat prompt template
# 6. create a stuff document chain that accepts a llm model and chat prompt template & we can also run stuff document chain by passing in documents directly

chat_prompt_template = ChatPromptTemplate.from_template(
"""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}"""
)

documents_chain = create_stuff_documents_chain(llm=llm, prompt=chat_prompt_template)
response = documents_chain.invoke(
    {
        "input": "how can langsmith help with testing?",
        "context": documents        
    }
)
print(response)

Based on the provided context, LangSmith can help with testing in the following ways:

1. **Creating datasets**: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications.
2. **Running custom evaluations**: LangSmith makes it easy to run custom evaluations (both LLM and heuristic-based) to score test results.
3. **Comparison view**: Langsmith provides a comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of the application.
4. **Playground environment**: The playground environment allows developers to quickly test out different prompts and models, with each run logged in the system and able to be used to create test cases or compare with other runs.
5. **Beta testing**: LangSmith supports beta testing by allowing developers to collect more data on how their LLM applications are performing in real-world scenarios, including feedback colle

## retrieval chain

In [27]:
# 7. create a document retrieval chain that takes vector store retriever and stuff document chain

retrieval_chain = create_retrieval_chain(vector_store_retriever, documents_chain) 
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})

# print(type(response))
pprint.pprint(response, indent=4)

{   'answer': 'Based on the provided context, LangSmith can help with testing '
              'in several ways:\n'
              '\n'
              '1. **Tracing**: LangSmith allows developers to create datasets, '
              'which are collections of inputs and reference outputs, and use '
              'these to run tests on their LLM applications.\n'
              '2. **Evaluation**: LangSmith provides native rendering of chat '
              'messages, functions, and retrieve documents, making it easy to '
              'evaluate the performance of an application.\n'
              '3. **Comparison View**: Langsmith offers a comparison view for '
              'test runs, allowing developers to track and diagnose '
              'regressions in test scores across multiple revisions of their '
              'application.\n'
              '4. **Playground**: The playground environment allows rapid '
              'iteration and experimentation, enabling developers to quickly '
    

## conversation retrieval chain

In [49]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])

history_aware_retriever_chain = create_history_aware_retriever(llm, vector_store_retriever, chat_prompt_template)

In [47]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

document_chain = create_stuff_documents_chain(llm, chat_prompt_template)
retrieval_chain = create_retrieval_chain(history_aware_retriever_chain, document_chain)

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]

response = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "tell me how"
})

pprint.pprint(response)

{'answer': 'LangSmith is designed to support various workflows throughout the '
           "development lifecycle of your LLM application. Here's an overview "
           'of how Langsmith can help you test your LLM applications:\n'
           '\n'
           '1. **Prototyping**: Langsmith provides a playground environment '
           'for rapid iteration and experimentation. You can quickly test out '
           'different prompts and models, and each run is logged in the '
           'system.\n'
           '2. **Debugging**: When things go wrong, Langsmith gives you clear '
           'visibility and debugging information at each step of an LLM '
           'sequence, making it easier to identify and root-cause issues.\n'
           '3. **Initial Test Set**: You can create datasets, which are '
           'collections of inputs and reference outputs, and use these to run '
           'tests on your LLM applications.\n'
           '4. **Comparison View**: Langsmith allows you to view